# Part 1: segmenting the cities in neighborhoods

Packages required

In [7]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
import requests

from sklearn.neighbors import KNeighborsClassifier

import folium

chicago neighborhoods

In [8]:
chicago_url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
source = requests.get(chicago_url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
body = table.find('tbody')
tsas = []
for tsa in table.find_all('tr'):
    tsas.append (tsa.get_text().splitlines()[1])
tsas = tsas [1:]
labels = ['Neighborhood']
chicago_neighborhoods = pd.DataFrame (tsas, columns=labels)

import geocoder
Latitudes=[]
Longitudes=[]
for i in range(chicago_neighborhoods.shape[0]):
    arg='{}, Chicago, Illinois'.format(chicago_neighborhoods.loc[i,'Neighborhood'])
    geopoint = geocoder.arcgis(arg)
    Latitudes.append(geopoint.latlng[0])
    Longitudes.append(geopoint.latlng[1])
    
chicago_neighborhoods['Latitude']=Latitudes
chicago_neighborhoods['Longitude']=Longitudes

ny neighborhoods

In [9]:
with open('ny_neighborhoods.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
ny_neighborhoods = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'])
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    ny_neighborhoods = ny_neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
ny_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Wakefield,40.894705,-73.847201
1,Co-op City,40.874294,-73.829939
2,Eastchester,40.887556,-73.827806
3,Fieldston,40.895437,-73.905643
4,Riverdale,40.890834,-73.912585


In [11]:
chicago_neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Albany Park,41.96829,-87.72338
1,Altgeld Gardens,41.65441,-87.60225
2,Andersonville,41.98046,-87.66834
3,Archer Heights,41.81154,-87.72556
4,Armour Square,41.83458,-87.63189


# Part 2: getting relevant data from Foursquare API

Foursquare credentials

In [12]:
CLIENT_ID='ZPLPGG3VSLT050LEEIARCWZ5TTPS0BNNV42WU3EBQVKQAMTF'
CLIENT_SECRET='DX5SON5DXJKGOXZXMX3R2D4Z1SVCPJKHXXNE42MEZ3PSHGQL'
VERSION='20190903'

method to get nearby venues

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

getting nearby venues for the neighborhoods in the 2 cities

In [14]:
ny_venues=getNearbyVenues(names=ny_neighborhoods['Neighborhood'], latitudes=ny_neighborhoods['Latitude'], longitudes=ny_neighborhoods['Longitude'])
chicago_venues=getNearbyVenues(names=chicago_neighborhoods['Neighborhood'], latitudes=chicago_neighborhoods['Latitude'], longitudes=chicago_neighborhoods['Longitude'])

In [15]:
ny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


one hot encoding per venue type

In [16]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
ny_onehot['Neighborhood'] = ny_venues['Neighborhood']

chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood']

# only keep categories present in both cities
fixed_columns = list(set(chicago_onehot.columns) & set(ny_onehot.columns))

ny_onehot = ny_onehot[fixed_columns]
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_total = ny_venues.groupby(['Neighborhood']).size().reset_index(name='counts')
ny_data = pd.merge(ny_grouped, ny_total, on='Neighborhood')

chicago_onehot = chicago_onehot[fixed_columns]
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_total = chicago_venues.groupby(['Neighborhood']).size().reset_index(name='counts')
chicago_data = pd.merge(chicago_grouped, chicago_total, on='Neighborhood')

In [17]:
chicago_data.head()

,Neighborhood,Speakeasy,Peruvian Restaurant,Gift Shop,Salon / Barbershop,Adult Boutique,Skate Park,Scenic Lookout,Train,Sake Bar,Diner,Dance Studio,Tanning Salon,Shop & Service,Arts & Crafts Store,Donut Shop,Baby Store,Thrift / Vintage Store,Ethiopian Restaurant,Fountain,Garden,Bank,Fruit & Vegetable Store,Supermarket,Camera Store,Turkish Restaurant,Storage Facility,Cosmetics Shop,Rental Service,Factory,Szechuan Restaurant,Trail,Italian Restaurant,Lingerie Store,Street Art,Brewery,Business Service,Clothing Store,Bike Trail,Mexican Restaurant,Hot Dog Joint,Massage Studio,Chocolate Shop,Burger Joint,New American Restaurant,Antique Shop,Hotel Pool,Fast Food Restaurant,Fried Chicken Joint,College Cafeteria,Playground,Art Gallery,Snack Place,Gymnastics Gym,Art Museum,Steakhouse,Jazz Club,Cocktail Bar,Other Great Outdoors,Gastropub,Breakfast Spot,Metro Station,Nail Salon,Intersection,Cuban Restaurant,Bubble Tea Shop,Beer Bar,Brazilian Restaurant,Baseball Stadium,Nightclub,Restaurant,Bus Stop,Greek Restaurant,Train Station,Kids Store,Sandwich Place,Park,Recreation Center,Track,Resort,Fish Market,Cupcake Shop,Gym / Fitness Center,Motorcycle Shop,Frozen Yogurt Shop,Home Service,Mini Golf,Sushi Restaurant,Bistro,Mattress Store,Bike Shop,BBQ Joint,Gym,Pilates Studio,Yoga Studio,Ice Cream Shop,Japanese Restaurant,Israeli Restaurant,Health Food Store,Insurance Office,Skating Rink,Laundromat,Food,Accessories Store,Arts & Entertainment,Rental Car Location,American Restaurant,Pub,Boat or Ferry,Bus Station,Department Store,Smoke Shop,Historic Site,Roof Deck,Video Store,Shoe Repair,German Restaurant,History Museum,Movie Theater,Pet Service,Wine Shop,Poke Place,Pool Hall,Vegetarian / Vegan Restaurant,Juice Bar,General Entertainment,Comic Shop,Shopping Mall,Bed & Breakfast,Bike Rental / Bike Share,Other Repair Shop,Record Shop,Creperie,Furniture / Home Store,Hardware Store,Pool,Multiplex,Moroccan Restaurant,Beer Store,Pharmacy,Dog Run,Men's Store,Dry Cleaner,Bar,Outdoors & Recreation,Gourmet Shop,Tattoo Parlor,Seafood Restaurant,Airport Terminal,Road,Jewelry Store,African Restaurant,Baseball Field,Lounge,Hostel,Malay Restaurant,Bookstore,Miscellaneous Shop,Indie Theater,Office,Indie Movie Theater,Moving Target,Flea Market,Supplement Shop,Toy / Game Store,Recording Studio,Comedy Club,Persian Restaurant,Bridge,Bakery,Museum,Convenience Store,Public Art,Heliport,Arcade,Sculpture Garden,Garden Center,Health & Beauty Service,Chinese Restaurant,Tiki Bar,Beach,Indonesian Restaurant,Tennis Court,Sports Bar,Vietnamese Restaurant,Pie Shop,Hawaiian Restaurant,Food Stand,Event Service,Electronics Store,Shoe Store,Czech Restaurant,Theater,Cafeteria,Pet Store,Burrito Place,Grocery Store,Deli / Bodega,Fish & Chips Shop,Stadium,Print Shop,Leather Goods Store,Discount Store,Indian Restaurant,Women's Store,Hookah Bar,Hobby Shop,Golf Course,Non-Profit,Gas Station,Taiwanese Restaurant,Wings Joint,Pizza Place,Halal Restaurant,Rock Club,Hotel Bar,Lake,Boutique,Farmers Market,Polish Restaurant,Bowling Alley,Athletics & Sports,Locksmith,Tapas Restaurant,Coffee Shop,Mediterranean Restaurant,Design Studio,Automotive Shop,Mobile Phone Shop,Food & Drink Shop,Dive Bar,Scandinavian Restaurant,South American Restaurant,Candy Store,Hotel,Plaza,Construction & Landscaping,Video Game Store,Liquor Store,Music Store,Caribbean Restaurant,Soccer Field,Food Truck,Molecular Gastronomy Restaurant,IT Services,Sporting Goods Shop,Cycle Studio,Bagel Shop,Flower Shop,Noodle House,Veterinarian,Outdoor Sculpture,Ramen Restaurant,Lawyer,Argentinian Restaurant,Karaoke Bar,Weight Loss Center,Bus Line,Concert Hall,Salad Place,Building,Eastern European Restaurant,Field,Farm,Latin American Restaurant,Event Space,Tea Room,Big Box Store,Asian Restaurant,Exhibit,Basketball Court,Community Center,Market,Monument / Landmark,Music Venue,Irish Pub,Dessert Shop,Optical Shop,Middle Eastern Restaurant,Climbing Gym,Comfort Food Restaurant,Tex-Mex Restaurant,Butcher,Taco Place,Performing Arts Venue,Food Court,G

In [18]:
ny_data.head()

,Neighborhood,Speakeasy,Peruvian Restaurant,Gift Shop,Salon / Barbershop,Adult Boutique,Skate Park,Scenic Lookout,Train,Sake Bar,Diner,Dance Studio,Tanning Salon,Shop & Service,Arts & Crafts Store,Donut Shop,Baby Store,Thrift / Vintage Store,Ethiopian Restaurant,Fountain,Garden,Bank,Fruit & Vegetable Store,Supermarket,Camera Store,Turkish Restaurant,Storage Facility,Cosmetics Shop,Rental Service,Factory,Szechuan Restaurant,Trail,Italian Restaurant,Lingerie Store,Street Art,Brewery,Business Service,Clothing Store,Bike Trail,Mexican Restaurant,Hot Dog Joint,Massage Studio,Chocolate Shop,Burger Joint,New American Restaurant,Antique Shop,Hotel Pool,Fast Food Restaurant,Fried Chicken Joint,College Cafeteria,Playground,Art Gallery,Snack Place,Gymnastics Gym,Art Museum,Steakhouse,Jazz Club,Cocktail Bar,Other Great Outdoors,Gastropub,Breakfast Spot,Metro Station,Nail Salon,Intersection,Cuban Restaurant,Bubble Tea Shop,Beer Bar,Brazilian Restaurant,Baseball Stadium,Nightclub,Restaurant,Bus Stop,Greek Restaurant,Train Station,Kids Store,Sandwich Place,Park,Recreation Center,Track,Resort,Fish Market,Cupcake Shop,Gym / Fitness Center,Motorcycle Shop,Frozen Yogurt Shop,Home Service,Mini Golf,Sushi Restaurant,Bistro,Mattress Store,Bike Shop,BBQ Joint,Gym,Pilates Studio,Yoga Studio,Ice Cream Shop,Japanese Restaurant,Israeli Restaurant,Health Food Store,Insurance Office,Skating Rink,Laundromat,Food,Accessories Store,Arts & Entertainment,Rental Car Location,American Restaurant,Pub,Boat or Ferry,Bus Station,Department Store,Smoke Shop,Historic Site,Roof Deck,Video Store,Shoe Repair,German Restaurant,History Museum,Movie Theater,Pet Service,Wine Shop,Poke Place,Pool Hall,Vegetarian / Vegan Restaurant,Juice Bar,General Entertainment,Comic Shop,Shopping Mall,Bed & Breakfast,Bike Rental / Bike Share,Other Repair Shop,Record Shop,Creperie,Furniture / Home Store,Hardware Store,Pool,Multiplex,Moroccan Restaurant,Beer Store,Pharmacy,Dog Run,Men's Store,Dry Cleaner,Bar,Outdoors & Recreation,Gourmet Shop,Tattoo Parlor,Seafood Restaurant,Airport Terminal,Road,Jewelry Store,African Restaurant,Baseball Field,Lounge,Hostel,Malay Restaurant,Bookstore,Miscellaneous Shop,Indie Theater,Office,Indie Movie Theater,Moving Target,Flea Market,Supplement Shop,Toy / Game Store,Recording Studio,Comedy Club,Persian Restaurant,Bridge,Bakery,Museum,Convenience Store,Public Art,Heliport,Arcade,Sculpture Garden,Garden Center,Health & Beauty Service,Chinese Restaurant,Tiki Bar,Beach,Indonesian Restaurant,Tennis Court,Sports Bar,Vietnamese Restaurant,Pie Shop,Hawaiian Restaurant,Food Stand,Event Service,Electronics Store,Shoe Store,Czech Restaurant,Theater,Cafeteria,Pet Store,Burrito Place,Grocery Store,Deli / Bodega,Fish & Chips Shop,Stadium,Print Shop,Leather Goods Store,Discount Store,Indian Restaurant,Women's Store,Hookah Bar,Hobby Shop,Golf Course,Non-Profit,Gas Station,Taiwanese Restaurant,Wings Joint,Pizza Place,Halal Restaurant,Rock Club,Hotel Bar,Lake,Boutique,Farmers Market,Polish Restaurant,Bowling Alley,Athletics & Sports,Locksmith,Tapas Restaurant,Coffee Shop,Mediterranean Restaurant,Design Studio,Automotive Shop,Mobile Phone Shop,Food & Drink Shop,Dive Bar,Scandinavian Restaurant,South American Restaurant,Candy Store,Hotel,Plaza,Construction & Landscaping,Video Game Store,Liquor Store,Music Store,Caribbean Restaurant,Soccer Field,Food Truck,Molecular Gastronomy Restaurant,IT Services,Sporting Goods Shop,Cycle Studio,Bagel Shop,Flower Shop,Noodle House,Veterinarian,Outdoor Sculpture,Ramen Restaurant,Lawyer,Argentinian Restaurant,Karaoke Bar,Weight Loss Center,Bus Line,Concert Hall,Salad Place,Building,Eastern European Restaurant,Field,Farm,Latin American Restaurant,Event Space,Tea Room,Big Box Store,Asian Restaurant,Exhibit,Basketball Court,Community Center,Market,Monument / Landmark,Music Venue,Irish Pub,Dessert Shop,Optical Shop,Middle Eastern Restaurant,Climbing Gym,Comfort Food Restaurant,Tex-Mex Restaurant,Butcher,Taco Place,Performing Arts Venue,Food Court,G

only keep neighborhoods with more than 20 venues

In [19]:
chicago_30 = chicago_data[chicago_data['counts'] > 30]
ny_30 = ny_data[ny_data['counts'] > 30]

In [20]:
chicago_30.set_index('Neighborhood', inplace=True)
ny_30.set_index('Neighborhood', inplace=True)

In [21]:
ny_30.head()

,Speakeasy,Peruvian Restaurant,Gift Shop,Salon / Barbershop,Adult Boutique,Skate Park,Scenic Lookout,Train,Sake Bar,Diner,Dance Studio,Tanning Salon,Shop & Service,Arts & Crafts Store,Donut Shop,Baby Store,Thrift / Vintage Store,Ethiopian Restaurant,Fountain,Garden,Bank,Fruit & Vegetable Store,Supermarket,Camera Store,Turkish Restaurant,Storage Facility,Cosmetics Shop,Rental Service,Factory,Szechuan Restaurant,Trail,Italian Restaurant,Lingerie Store,Street Art,Brewery,Business Service,Clothing Store,Bike Trail,Mexican Restaurant,Hot Dog Joint,Massage Studio,Chocolate Shop,Burger Joint,New American Restaurant,Antique Shop,Hotel Pool,Fast Food Restaurant,Fried Chicken Joint,College Cafeteria,Playground,Art Gallery,Snack Place,Gymnastics Gym,Art Museum,Steakhouse,Jazz Club,Cocktail Bar,Other Great Outdoors,Gastropub,Breakfast Spot,Metro Station,Nail Salon,Intersection,Cuban Restaurant,Bubble Tea Shop,Beer Bar,Brazilian Restaurant,Baseball Stadium,Nightclub,Restaurant,Bus Stop,Greek Restaurant,Train Station,Kids Store,Sandwich Place,Park,Recreation Center,Track,Resort,Fish Market,Cupcake Shop,Gym / Fitness Center,Motorcycle Shop,Frozen Yogurt Shop,Home Service,Mini Golf,Sushi Restaurant,Bistro,Mattress Store,Bike Shop,BBQ Joint,Gym,Pilates Studio,Yoga Studio,Ice Cream Shop,Japanese Restaurant,Israeli Restaurant,Health Food Store,Insurance Office,Skating Rink,Laundromat,Food,Accessories Store,Arts & Entertainment,Rental Car Location,American Restaurant,Pub,Boat or Ferry,Bus Station,Department Store,Smoke Shop,Historic Site,Roof Deck,Video Store,Shoe Repair,German Restaurant,History Museum,Movie Theater,Pet Service,Wine Shop,Poke Place,Pool Hall,Vegetarian / Vegan Restaurant,Juice Bar,General Entertainment,Comic Shop,Shopping Mall,Bed & Breakfast,Bike Rental / Bike Share,Other Repair Shop,Record Shop,Creperie,Furniture / Home Store,Hardware Store,Pool,Multiplex,Moroccan Restaurant,Beer Store,Pharmacy,Dog Run,Men's Store,Dry Cleaner,Bar,Outdoors & Recreation,Gourmet Shop,Tattoo Parlor,Seafood Restaurant,Airport Terminal,Road,Jewelry Store,African Restaurant,Baseball Field,Lounge,Hostel,Malay Restaurant,Bookstore,Miscellaneous Shop,Indie Theater,Office,Indie Movie Theater,Moving Target,Flea Market,Supplement Shop,Toy / Game Store,Recording Studio,Comedy Club,Persian Restaurant,Bridge,Bakery,Museum,Convenience Store,Public Art,Heliport,Arcade,Sculpture Garden,Garden Center,Health & Beauty Service,Chinese Restaurant,Tiki Bar,Beach,Indonesian Restaurant,Tennis Court,Sports Bar,Vietnamese Restaurant,Pie Shop,Hawaiian Restaurant,Food Stand,Event Service,Electronics Store,Shoe Store,Czech Restaurant,Theater,Cafeteria,Pet Store,Burrito Place,Grocery Store,Deli / Bodega,Fish & Chips Shop,Stadium,Print Shop,Leather Goods Store,Discount Store,Indian Restaurant,Women's Store,Hookah Bar,Hobby Shop,Golf Course,Non-Profit,Gas Station,Taiwanese Restaurant,Wings Joint,Pizza Place,Halal Restaurant,Rock Club,Hotel Bar,Lake,Boutique,Farmers Market,Polish Restaurant,Bowling Alley,Athletics & Sports,Locksmith,Tapas Restaurant,Coffee Shop,Mediterranean Restaurant,Design Studio,Automotive Shop,Mobile Phone Shop,Food & Drink Shop,Dive Bar,Scandinavian Restaurant,South American Restaurant,Candy Store,Hotel,Plaza,Construction & Landscaping,Video Game Store,Liquor Store,Music Store,Caribbean Restaurant,Soccer Field,Food Truck,Molecular Gastronomy Restaurant,IT Services,Sporting Goods Shop,Cycle Studio,Bagel Shop,Flower Shop,Noodle House,Veterinarian,Outdoor Sculpture,Ramen Restaurant,Lawyer,Argentinian Restaurant,Karaoke Bar,Weight Loss Center,Bus Line,Concert Hall,Salad Place,Building,Eastern European Restaurant,Field,Farm,Latin American Restaurant,Event Space,Tea Room,Big Box Store,Asian Restaurant,Exhibit,Basketball Court,Community Center,Market,Monument / Landmark,Music Venue,Irish Pub,Dessert Shop,Optical Shop,Middle Eastern Restaurant,Climbing Gym,Comfort Food Restaurant,Tex-Mex Restaurant,Butcher,Taco Place,Performing Arts Venue,Food Court,Gay Bar,Gym Po

separating x and y (the relative amount of pizzerias is the value to be predicted y)

In [22]:
chicago_Y = chicago_30['Pizza Place']
chicago_X = chicago_30
del chicago_X['Pizza Place']

In [23]:
chicago_counts=chicago_X['counts']

# Part 3: training the model on Chicago

normalization

In [24]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(chicago_X).transform(chicago_X.astype(float))

separating train and test data

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (X, chicago_Y, test_size = 0.3, random_state = 0)

find best K for a K nearest neighbors regressor, weighting them with the distance to the predictor

In [26]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

Ks = 10
mse = np.zeros((Ks-1))
for n in range(1,Ks):
    neigh = KNeighborsRegressor(n_neighbors = n, weights='distance').fit(x_train,y_train)
    yhat=neigh.predict(x_test)
    mse[n-1] = mean_squared_error(y_test, yhat)
print ("best K is %d with MSE %f" % (mse.argmin()+1, mse.min()))

best K is 5 with MSE 0.000316


In [27]:
meanY = y_test.mean()
print ('the mean frequency of pizza places in the test set is %f, so the MSE is %f %% of the mean frequency'
      % (meanY, 100*mse.min()/meanY))

the mean frequency of pizza places in the test set is 0.034039, so the MSE is 0.929583 % of the mean frequency


which means the model has great accuracy

In [28]:
myModel = KNeighborsRegressor(n_neighbors=5, weights='distance').fit(X,chicago_Y)

# Part 4: predictions on the new city and comparisons

prepare the dataset

In [29]:
ny_Y = ny_30['Pizza Place']
ny_X = ny_30
del ny_X['Pizza Place']

ny_counts=ny_X['counts']

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(ny_X).transform(ny_X.astype(float))

predict

In [30]:
y_hat = myModel.predict(X)

create dataset with all useful information

In [31]:
ny_Y = pd.DataFrame(ny_Y)

In [32]:
NY_final = pd.merge(ny_total, ny_Y, on='Neighborhood')

In [33]:
NY_final['predicted'] = y_hat

In [34]:
NY_final.head()

,Neighborhood,counts,Pizza Place,predicted
0,Astoria,100,0.020000,0.025234
1,Bath Beach,45,0.022222,0.051870
2,Battery Park City,100,0.020000,0.033969
3,Bay Ridge,85,0.035294,0.064577
4,Bay Terrace,50,0.020000,0.038475


accuracy score

In [35]:
meanY = ny_Y.mean()
mse = mean_squared_error (ny_Y, y_hat)
print ('the mean frequency of pizza places in the ny set is %f, so the MSE is %f %% of the mean frequency'
      % (meanY, 100*mse.min()/meanY))

the mean frequency of pizza places in the ny set is 0.040289, so the MSE is 2.651075 % of the mean frequency


add differences between prediction and actual frequency to the dataset

In [36]:
NY_final['Frequency difference']=NY_final['predicted']-NY_final['Pizza Place']
NY_final['Absolute difference']=NY_final['Frequency difference']*NY_final['counts']

In [37]:
NY_final.head()

,Neighborhood,counts,Pizza Place,predicted,Frequency difference,Absolute difference
0,Astoria,100,0.020000,0.025234,0.005234,0.523362
1,Bath Beach,45,0.022222,0.051870,0.029648,1.334152
2,Battery Park City,100,0.020000,0.033969,0.013969,1.396940
3,Bay Ridge,85,0.035294,0.064577,0.029283,2.489081
4,Bay Terrace,50,0.020000,0.038475,0.018475,0.923740


look for a bias in the predictions, which could be due to differences between the two cities

In [38]:
print ('The mean absolute difference between predicted amount of Pizza Places and actual amount is: %f' 
       %NY_final['Absolute difference'].mean())

The mean absolute difference between predicted amount of Pizza Places and actual amount is: 0.011043


This means New York tends to have a slightly higher relative amount of Pizza Places.
It's not a big problem: it means that the predictions are a bit pessimistic.

# Part 6: visualize the results on a Folium map and final results

add back the coordinates to the dataset

In [39]:
NY_final.set_index('Neighborhood', inplace=True)

In [40]:
ny_neighborhoods.set_index('Neighborhood', inplace=True)

In [41]:
NY = pd.merge(ny_neighborhoods, NY_final, on='Neighborhood', how='inner')

In [42]:
NY.head()

,Latitude,Longitude,counts,Pizza Place,predicted,Frequency difference,Absolute difference
Neighborhood,,,,,,,
Kingsbridge,40.881687,-73.902818,67,0.074627,0.056290,-0.018337,-1.228573
Norwood,40.877224,-73.879391,32,0.156250,0.037421,-0.118829,-3.802536
Bedford Park,40.870185,-73.885512,41,0.073171,0.062411,-0.010760,-0.441169
Fordham,40.860997,-73.896427,85,0.058824,0.057590,-0.001234,-0.104864
Parkchester,40.837938,-73.856003,35,0.085714,0.056810,-0.028905,-1.011662


visualize

In [43]:
import folium

In [44]:
NY.reset_index(inplace=True)

In [45]:
venues_map = folium.Map(location=[40.730610, -73.935242], zoom_start=10)

def getColor (difference):
    if difference < -1:
        return 'darkred'
    elif difference < 0:
        return 'red'
    elif difference < 1:
        return 'orange'
    elif difference < 2:
        return 'lightgreen'
    else:
        return 'darkgreen'

for i in range (0, len(NY)):
    folium.CircleMarker([NY.loc[i, 'Latitude'], NY.loc[i, 'Longitude']], 
                        radius=5, color=getColor(NY.loc[i, 'Absolute difference']),
                        popup='{} : there seems to be place for {} Pizza places more'.format(NY.loc[i, 'Neighborhood'], NY.loc[i, 'Absolute difference']),
                        fill = True, fill_color = getColor(NY.loc[i, 'Absolute difference']), fill_opacity = 0.6).add_to(venues_map)

In [46]:
venues_map

find best neighborhoods to start a pizza place in New York

In [47]:
NY.sort_values(by='Absolute difference', axis=0, ascending=False, inplace=True)

In [48]:
NY

,Neighborhood,Latitude,Longitude,counts,Pizza Place,predicted,Frequency difference,Absolute difference
54,Greenwich Village,40.726933,-73.999914,100,0.010000,0.053854,0.043854,4.385410
106,Sunnyside Gardens,40.745652,-73.918193,100,0.030000,0.068807,0.038807,3.880698
37,Washington Heights,40.851903,-73.936900,85,0.023529,0.068738,0.045209,3.842752
102,Hunters Point,40.743414,-73.953868,75,0.013333,0.057851,0.044518,3.338857
32,East Williamsburg,40.708492,-73.938858,69,0.000000,0.045870,0.045870,3.165022
53,Chelsea,40.594726,-74.189560,105,0.009524,0.038717,0.029193,3.065236
52,Chelsea,40.744035,-74.003116,105,0.009524,0.038717,0.029193,3.065236
60,West Village,40.734434,-74.006180,100,0.020000,0.049608,0.029608,2.960769
95,Noho,40.723259,-73.988434,100,0.030000,0.058823,0.028823,2.882336
7,Pelham Bay,40.850641,-73.832074,44,0.000000,0.062416,0.062416,2.746317
